In [1]:
def spec(M,d):
    l = []
    
    if d == 1:
        return [M]
    else:
        a0list = []
        for a0 in range(1, int(M/2) +1):
            if M % a0 == 0:
                a0list.append(a0)
        new = []
        for a0 in a0list:
            
            slist = spec(int(M/a0)-1, d-1)
            if d ==2:
                new = new + [[a0] + slist]
            else:
                for s in slist:
                    new.append([a0] + s)
      
        return new
    
def summer(P):
    if len(P) == 1:
        return P[0]
    else:
        x = P[0]
        return x + x*summer(P[1:])

def prod(l):
    #return product of a list
    p = 1
    for i in l:
        p = p*i
    return p

def make_tree_dict(P):
    #return a dictionary representing the tree structure of P
    
    #summer(P) is runs the depth equation on P.
    M = summer(P)
    
    #This loop creates an empty dictionary with M items
    d = {}
    for i in range(0,M):
        d[i] = []
        
    if len(P) == 1:
        return d
    
    #The current node to assign children to, and the starting
    #index for children.
    current = 0    
    nextnode = P[0]
    
    #Set the starting level and pernode
    level = 1
    pernode = P[level]
    
    #The maximum number of nodes in the first tier
    levelmax = P[0]
    
    #block for entire list
    todo = M - prod(P) 
    while current < todo :
        inlevel = 0
        pernode = P[level]
        
        #block for each tier
        while inlevel < levelmax  and current < todo:
                    
            #This block account for the children of a specific node
            count = 0
            while count < pernode :
                d[current] += [nextnode]
                nextnode += 1
                count += 1
            current += 1
            inlevel += 1
        
        #Once a tier is finished, move on to next, update levelmax appropriately
        # for the next tier
        level += 1        
        levelmax = inlevel * pernode        
            
    return d

def make_tree(P):
    d = make_tree_dict(P)
    return DiGraph(d)

def genIdeal(x, D):
    #given element x and tree D, return ideal generated by x in D as a set
    if D.neighbors_out(x) == []:
        return set([x])
    else:
        s = set([x])
        for i in D.neighbors_out(x):
            s = s.union(genIdeal(i,D))
        return s

In [2]:
def cartUnion(L1, L2):
    #cartesion product analogue for list of lists
    if L1 == []:
        return L2
    
    
    res = []    
    for l1 in L1:
        for l2 in L2:
            res += [l1 + l2]
    return res

In [3]:
cartUnion([[5],[6]],[[3],[4]])

[[5, 3], [5, 4], [6, 3], [6, 4]]

In [4]:
def ideals_onetree(D, root, t):
    #return the set of ideals in D of size t, Assuming D is a connected tree.
    result = []
    if t == 0 or t > len(D.vertices()):
        return []
    if len(D.vertices()) == t:
    #first check if the entire tree is an ideal of size t
        return [D.vertices()]
    
    else:            
        #A list of top row elements
        toprow = D.neighbors_out(root) 
        
        #generate the list of weak compositions of size t for a0 many trees
        #apparently using min_part=0 may produce errors. May try to reverse engineer
        #in the future.
        weakcomps = Compositions(t, length = len(toprow), min_part=0).list()

        for comp in weakcomps:
            #for each weak composition find ideals for each subtree
            #with sizes according to the composition. Concatenating
            #these lists should produce an ideal of size t if theory
            #is correct           
            
            compideal = []
            for i in range(len(comp)):
                if comp[i] != 0:
                    subVerts = list(genIdeal(toprow[i],D))
                    subTree = D.subgraph(subVerts)
                    compideal = cartUnion(compideal, ideals_onetree(subTree, toprow[i], comp[i]))
            if compideal != [] and len(compideal[0]) == t:        
                result += compideal
        return result  

In [5]:
ideals_onetree(make_tree([1,2,1]), 0,2)

/opt/sagemath-9.2/local/lib/python3.7/site-packages/sage/combinat/composition.py:1601: RuntimeWarning: Currently, setting min_part=0 produces Composition objects which violate internal assumptions.  Calling methods on these objects may produce errors or WRONG results!
  warn("Currently, setting min_part=0 produces Composition objects which violate internal assumptions.  Calling methods on these objects may produce errors or WRONG results!", RuntimeWarning)


[[1, 3], [3, 4], [2, 4]]

In [12]:
def dummyRoot(D, a0):
    #return a tree with dummy root 
    D.add_vertex(-1)
    toprow = [i for i in range(a0)]
    for node in toprow:
        D.add_edge(-1,node)
    return D

def allideals_manytrees(D, a0, t):
    if len(D.vertices()) == t:
    #first check if the entire tree is an ideal of size t
        return [D.vertices()]
    else:
        D = dummyRoot(D,a0)
        return ideals_onetree(D, -1, t)

In [14]:
allideals_manytrees(make_tree([2,2,1,2]), 2, 12)

[[0, 2, 3, 6, 7, 10, 11, 12, 13, 8, 14, 15],
 [0, 2, 3, 6, 7, 10, 11, 12, 13, 14, 15, 16],
 [0, 2, 3, 6, 7, 10, 11, 12, 13, 14, 15, 17],
 [0, 2, 3, 6, 7, 10, 11, 12, 13, 14, 16, 17],
 [0, 2, 3, 6, 7, 10, 11, 12, 13, 15, 16, 17],
 [0, 2, 3, 6, 7, 10, 11, 12, 13, 9, 16, 17],
 [2, 6, 10, 11, 3, 7, 12, 13, 4, 8, 14, 15],
 [2, 6, 10, 11, 3, 7, 12, 13, 8, 14, 15, 16],
 [2, 6, 10, 11, 3, 7, 12, 13, 8, 14, 15, 17],
 [2, 6, 10, 11, 3, 7, 12, 13, 14, 15, 16, 17],
 [2, 6, 10, 11, 3, 7, 12, 13, 14, 9, 16, 17],
 [2, 6, 10, 11, 3, 7, 12, 13, 15, 9, 16, 17],
 [2, 6, 10, 11, 3, 7, 12, 13, 5, 9, 16, 17],
 [2, 6, 10, 11, 7, 12, 13, 4, 8, 14, 15, 16],
 [2, 6, 10, 11, 7, 12, 13, 4, 8, 14, 15, 17],
 [2, 6, 10, 11, 7, 12, 13, 8, 14, 15, 16, 17],
 [2, 6, 10, 11, 7, 12, 13, 14, 15, 9, 16, 17],
 [2, 6, 10, 11, 7, 12, 13, 14, 5, 9, 16, 17],
 [2, 6, 10, 11, 7, 12, 13, 15, 5, 9, 16, 17],
 [6, 10, 11, 3, 7, 12, 13, 4, 8, 14, 15, 16],
 [6, 10, 11, 3, 7, 12, 13, 4, 8, 14, 15, 17],
 [6, 10, 11, 3, 7, 12, 13, 8, 14, 1

In [8]:
import time

In [20]:
P = [2,1,2,2,2]
t0 = time.time()
x =allideals_manytrees(make_tree(P),2,4)
t1 = time.time()
print("Compositions version  {} seconds.".format(t1-t0))
print(len(x))

Compositions version  0.09701776504516602 seconds.
1932
